In [1]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

import warnings
warnings.filterwarnings("ignore")

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# parameters
# state = Colorado
state_ab = "co"

# Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [3]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2021/2021_Approved_Congressional_Plan_w_Final_Adjustments.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2021/2021_Approved_Senate_Plan_w_Final_Adjustments.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2021/2021_Approved_House_Plan_w_Final_Adjustments.shp".format(data_folder, state_ab)

In [4]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df, disconnection_threshold=0, snap_precision=8)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [25]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [6]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [23]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [24]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [8]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [9]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [10]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [11]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [12]:
population_df.rename(columns=rename_dict, inplace = True)

In [13]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [15]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [16]:
vest20 = gpd.read_file(vest20_data)
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|████████████████████████████████████████| 3215/3215 [00:11<00:00, 275.89it/s]


There are 1436 overlaps.
There are 8880 holes.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 11234/11234 [00:07<00:00, 1457.96it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to fill: 100%|█████████████████████████████| 528/528 [05:04<00:00,  1.73it/s]


100%|████████████████████████████████████████| 3215/3215 [00:09<00:00, 331.88it/s]


In [17]:
vest20.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'PRECINCT', 'G20PREDBID',
       'G20PRERTRU', 'G20PRELJOR', 'G20PREGHAW', 'G20PRECBLA', 'G20PREUWES',
       'G20PREOOTH', 'G20USSDHIC', 'G20USSRGAR', 'G20USSLDOA', 'G20USSODOY',
       'G20USSOEVA', 'G20USSOWRI', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [18]:
start_col = 5
vest_base_data = vest20
year = '20'

In [19]:
vest_base = add_vest_base(vest_base_data, start_col, year)

# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 3215/3215 [01:13<00:00, 43.70it/s]


In [20]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [21]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

In [26]:
check_population(population_df, vest_base)

population agrees


# Add more vest data
### vest 18

In [27]:
vest18 = gpd.read_file(vest18_data)
vest18.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'PRECINCT', 'G18GOVDPOL',
       'G18GOVRSTA', 'G18GOVLHEL', 'G18GOVOHAM', 'G18ATGDWEI', 'G18ATGRBRA',
       'G18ATGLROB', 'G18SOSDGRI', 'G18SOSRWIL', 'G18SOSCCAM', 'G18SOSOHUB',
       'G18TREDYOU', 'G18TRERWAT', 'G18TRECKIL', 'G18RGTDSMI', 'G18RGTRMON',
       'G18RGTLTRE', 'G18RGTOOTW', 'geometry'],
      dtype='object')

In [28]:
# check the result here
start_col = 5
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|████████████████████████████████████████| 3136/3136 [00:10<00:00, 289.98it/s]


There are 1287 overlaps.
There are 5567 holes.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 10305/10305 [00:06<00:00, 1544.64it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to fill: 100%|█████████████████████████████| 522/522 [06:03<00:00,  1.43it/s]


100%|█████████████████████████████████████████| 3215/3215 [02:19<00:00, 23.09it/s]


population agrees


### vest 16

In [29]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAME', 'PRECINCT', 'G16PREDCLI',
       'G16PRERTRU', 'G16PRELJOH', 'G16PREGSTE', 'G16PREUMCM', 'G16PREOOTH',
       'G16USSDBEN', 'G16USSRGLE', 'G16USSLTAN', 'G16USSGMEN', 'G16USSOOTH',
       'G16RGTDMAD', 'G16RGTRGAN', 'geometry'],
      dtype='object')

In [30]:
start_col = 5
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 3010/3010 [00:11<00:00, 268.27it/s]


There are 1889 overlaps.
There are 8779 holes.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████| 14347/14347 [00:09<00:00, 1503.73it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 5055it [42:04,  2.00it/s]                                       
Gaps to fill: 100%|█████████████████████████████| 714/714 [07:09<00:00,  1.66it/s]


100%|█████████████████████████████████████████| 3215/3215 [01:13<00:00, 44.01it/s]


population agrees


## Add the district data

In [31]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [32]:
cong_df.head()

,OBJECTID,District,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,1,1,2.495876,7.814496e+05,4.336640e+09,"POLYGON ((-104.66853 39.90689, -104.65916 39.9..."
1,2,2,11.320395,3.521568e+06,3.218254e+11,"POLYGON ((-106.97868 41.00338, -106.97421 41.0..."
2,3,3,23.731473,7.604425e+06,1.396837e+12,"POLYGON ((-107.36708 41.00307, -107.36565 41.0..."
3,4,4,19.406221,6.234628e+06,9.005584e+11,"POLYGON ((-102.59825 41.00242, -102.58208 41.0..."
4,5,5,3.028264,9.616973e+05,4.109716e+10,"POLYGON ((-104.91224 39.12984, -104.91220 39.1..."


In [33]:
send.head()

,OBJECTID,District,Shape_Leng,Shape_Area,geometry
0,1,1,3.080656e+06,3.564080e+11,"MULTIPOLYGON (((-104.75617 40.38518, -104.7560..."
1,2,2,5.278787e+05,3.252466e+09,"POLYGON ((-104.79284 39.56609, -104.79137 39.5..."
2,3,3,1.099216e+06,6.685520e+10,"POLYGON ((-104.07353 38.52243, -104.07248 38.5..."
3,4,4,2.995556e+06,2.069199e+11,"POLYGON ((-105.16784 39.62868, -105.16694 39.6..."
4,5,5,3.619013e+06,2.167483e+11,"POLYGON ((-107.11386 39.62420, -107.11369 39.6..."


In [34]:
hdist.head()

,OBJECTID,District,Shape_Leng,Shape_Area,geometry
0,1,1,221814.501000,4.541396e+08,"POLYGON ((-105.03404 39.70224, -105.03283 39.7..."
1,2,2,93555.504961,3.488605e+08,"POLYGON ((-104.96019 39.72911, -104.96019 39.7..."
2,3,3,191869.674113,5.994867e+08,"POLYGON ((-105.01570 39.67868, -105.01340 39.6..."
3,4,4,101223.744383,3.231791e+08,"POLYGON ((-105.02512 39.79111, -105.02511 39.7..."
4,5,5,135698.713049,4.272490e+08,"POLYGON ((-104.96874 39.79558, -104.96874 39.7..."


In [36]:
election_df = add_district(cong_df, "CD", election_df, "District")

100%|███████████████████████████████████████████████| 8/8 [00:00<00:00, 21.94it/s]


There are 1 overlaps.
There are 1 holes.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 308.08it/s]


Resolving overlaps...
Filling gaps...


Gaps to simplify: 100%|█████████████████████████████| 4/4 [00:02<00:00,  1.98it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|███████████████████████████████████████████████| 8/8 [00:12<00:00,  1.57s/it]


In [37]:
election_df = add_district(send, "SEND", election_df, "District")

100%|█████████████████████████████████████████████| 35/35 [00:02<00:00, 13.02it/s]


There are 9 overlaps.
There are 12 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|████████████████████████████████████████████| 39/39 [00:00<00:00, 336.53it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 35/35 [00:10<00:00,  3.40it/s]


In [38]:
election_df = add_district(hdist, "HDIST", election_df, "District")

100%|█████████████████████████████████████████████| 65/65 [00:01<00:00, 59.90it/s]


There are 13 overlaps.
There are 25 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|████████████████████████████████████████████| 70/70 [00:00<00:00, 464.47it/s]

Resolving overlaps...
Assigning order 2 pieces...


Filling gaps...


100%|█████████████████████████████████████████████| 65/65 [00:10<00:00,  6.25it/s]


In [40]:
list(election_df.columns)

['2MOREVAP',
 'AMINVAP',
 'ASIANVAP',
 'ATG18D',
 'ATG18O',
 'ATG18R',
 'BVAP',
 'COUNTYFP',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'NAME',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'PRECINCT',
 'RGT16D',
 'RGT16R',
 'RGT18D',
 'RGT18O',
 'RGT18R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'STATEFP',
 'TOTPOP',
 'TRE18D',
 'TRE18O',
 'TRE18R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS20D',
 'USS20O',
 'USS20R',
 'VAP',
 'VTDST',
 'WVAP',
 'geometry',
 'CD',
 'SEND',
 'HDIST']

In [55]:
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'PRECINCT':'PRECINCT'+year,
        'GEOID':'GEOID'+year,
        'NAME':'NAME'+year,
        'VTDST':'VTDST'+year
    }
election_df.rename(columns=base_columns, inplace = True)

In [56]:
# reorder the columns

fixed_columns = [
    'STATEFP'+year,
    'COUNTYFP'+year,
    'PRECINCT'+year,
    'VTDST'+year,
    # 'GEOID'+year,
    'NAME'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [57]:
list(election_df.columns)

['STATEFP20',
 'COUNTYFP20',
 'PRECINCT20',
 'VTDST20',
 'NAME20',
 'CD',
 'SEND',
 'HDIST',
 'TOTPOP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'HISP',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'H_2MORE',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'ATG18D',
 'ATG18O',
 'ATG18R',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'RGT16D',
 'RGT16R',
 'RGT18D',
 'RGT18O',
 'RGT18R',
 'SOS18D',
 'SOS18O',
 'SOS18R',
 'TRE18D',
 'TRE18O',
 'TRE18R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS20D',
 'USS20O',
 'USS20R',
 'geometry']

In [59]:
# store the result in directory "{state abbreviation}"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))